In [14]:
import pickle

# path = "/data/sabri/cartridges/2025-08-05-10-00-14-m08d05_longhealth_synthesize_ntp/m08d05_longhealth_synthesize_ntp_llama-3.2-3b_p10_n65536-0/artifact/dataset.pkl"
path = "/data/sabri/cartridges/2025-08-05-09-43-20-m07d11_longhealth_synthesize/m07d11_longhealth_synthesize_llama-3.2-3b_p10_n65536-0/artifact/dataset.pkl"

with open(path, "rb") as f:
    dataset = pickle.load(f)

rows = dataset["rows"]

In [26]:
import numpy as np
import pandas as pd
from tqdm import tqdm

logprobs = []
token_idxs = []

curr_pos = 0
for row in tqdm(rows[:1024]):
    for message in row.messages:
        if message.top_logprobs is None:
            continue
        token_idxs.append(message.top_logprobs.token_idx + curr_pos)
        logprobs.append(message.top_logprobs.logprobs)
        curr_pos += message.top_logprobs.shape[0]

token_idxs = np.concatenate(token_idxs)
logprobs = np.concatenate(logprobs)

df = pd.DataFrame({
    "token_idx": token_idxs,
    "logprob": logprobs,
})

df.head()
        

100%|██████████| 1024/1024 [00:00<00:00, 113749.86it/s]


,token_idx,logprob
0,0,-0.327527
1,0,-1.327527
2,0,-4.452528
3,1,-0.000039
4,2,-0.000818


In [27]:
df["prob"] = np.exp(df["logprob"])

In [35]:
(df.groupby("token_idx")["prob"].max() > 0.995).mean()

np.float64(0.9874641813439262)